# <center>Machine Learning Project Code</center>

<a class="anchor" id="top"></a>

## <center>*02 - Holdout Method*</center>

** **



# Table of Contents  <br>


1. [Importing Libraries & Data](#1.-Importing-Libraries-&-Data) <br><br>
    
2. [Train-Test Split](#2.-Train-Test-Split)

   2.1 [Feature Engineering](#2.1-Feature-Engineering) <br>
    
   2.2 [Missing Values](#2.2-Missing-Values) <br>
    
   2.3 [Outliers](#2.3-Outliers) <br>
   
   2.4 [Scaling](#2.4-Scaling) <br><br>
   
3. [Feature Selection](#3.-Feature-Selection) 
    
    3.1 [Filter Based Methods](#3.1-Filter-Based-Methods) <br>

    3.2 [Wrapper Methods](#3.2-Wrapper-Methods) <br>
    
    3.3 [Embedded Methods](#3.3-Embedded-Methods) <br><br>
    
4. [Modeling](#4.-Modeling) <br><br>

5. [Export](#5.-Export)


** **

In this notebook we will start by spliting train and validation data using a simple Holdout Method. After doing so, more complex methods to fill missing values will be applied, and outliers will be addresses. Feature Selection will also be performed, followed by a Modeling section.

Data Scientist Manager: António Oliveira, **20211595**

Data Scientist Senior: Tomás Ribeiro, **20240526**

Data Scientist Junior: Gonçalo Pacheco, **20240695**

Data Analyst Senior: Gonçalo Custódio, **20211643**

Data Analyst Junior: Ana Caleiro, **20240696**


** ** 

# 1. Importing Libraries & Data
In this section, we set up the foundation for our project by importing the necessary Python libraries and loading the dataset. These libraries provide the tools for data manipulation, visualization, and machine learning modeling throughout the notebook. Additionally, we import the historical claims dataset, which forms the core of our analysis. 

In [42]:
import pandas as pd
import numpy as np

import utils2 as p

# Train-Test Split
from sklearn.model_selection import train_test_split

# Feature Selection
import feature_selection as fs

# Models
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, \
    GradientBoostingClassifier, AdaBoostClassifier, \
    StackingClassifier, VotingClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier  
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import RobustScaler

# Metrics
import metrics as m

pd.set_option('display.max_columns', None)
# Suppress Warnings
import warnings
warnings.filterwarnings("ignore")

**Import Data**

In [ ]:
# Load training data
df = pd.read_csv('./train_data_EDA.csv', index_col = 'Claim Identifier')

# Load testing data
test = pd.read_csv('./test_data_EDA.csv', index_col = 'Claim Identifier')

# Display the first 3 rows of the training data
df.head(3)

# 2. Train-Test Split
The train-test split is a crucial technique used to assess model performance by dividing the dataset into training and testing subsets. This ensures that the model is evaluated on unseen data, helping to prevent overfitting and providing an unbiased performance estimate. 

<a href="#top">Top &#129033;</a>

**Holdout Method**

In [21]:
# Split the DataFrame into features (X) and target variable (y)
X = df.drop('Claim Injury Type', axis=1) 
y = df['Claim Injury Type']  

In [22]:
# Split the dataset into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, 
                                                    test_size=0.2, 
                                                    random_state=42,
                                                    stratify = y) 

## 2.1 Feature Engineering

<a href="#top">Top &#129033;</a>

### 2.1.1 Encoding

Encoding transforms categorical data into numerical format for use in machine learning models. For this section, several encoders were considered:
- **One Hot Encoding** -  turns a variable that is stored in a column into dummy variables stored over multiple columns and represented as 0s and 1s
- **Frequency Encoding** - replaces the categories by with their proportion in the dataset
- **Count Encoding** - replaces the categories by the number of times they appear in the dataset 
- **Manual Mapping Encoding** - manually attribute values to each category

**Alternative Dispute Resolution**

Knowing that 'N' is by far the most common category, and that 'U' only appears 5 times in DF data and 1 time in the test data, we decided to join 'U' and 'Y' into 'Y/U', and encode the variable as:
- 0 - N
- 1 - Y/U

In [ ]:
print(X_train['Alternative Dispute Resolution Bin'].value_counts())
print(' ')
print(test['Alternative Dispute Resolution Bin'].value_counts())

In [ ]:
X_train['Alternative Dispute Resolution'] = X_train['Alternative Dispute Resolution Bin'].replace({'N': 0, 'Y': 1, 'U': 1})
X_val['Alternative Dispute Resolution'] = X_val['Alternative Dispute Resolution Bin'].replace({'N': 0, 'Y': 1, 'U': 1})
test['Alternative Dispute Resolution'] = test['Alternative Dispute Resolution Bin'].replace({'N': 0, 'Y': 1, 'U': 1})

**Attorney/Representative**

As this variable only has 2 categories, they will be encoded as follows:
- N - 0
- Y - 1

In [ ]:
print(X_train['Attorney/Representative Bin'].value_counts())
print(' ')
print(test['Attorney/Representative Bin'].value_counts())

In [30]:
X_train['Attorney/Representative'] = X_train['Attorney/Representative Bin'].replace({'N': 0, 'Y': 1})
X_val['Attorney/Representative'] = X_val['Attorney/Representative Bin'].replace({'N': 0, 'Y': 1})
test['Attorney/Representative'] = test['Attorney/Representative Bin'].replace({'N': 0, 'Y': 1})

**Carrier Name**

As Carrier name has a considerable amount of unique values, it will be encoded using Count Encoder.

We will start by analysing the common Carriers between train and test sets

In [31]:
train_carriers = set(X_train['Carrier Name Enc'].unique())
test_carriers = set(test['Carrier Name Enc'].unique())

common_categories = train_carriers.intersection(test_carriers)

Then map the common categories to an index

In [32]:
common_category_map = {category: idx + 1 for idx, 
                       category in enumerate(common_categories)}

Fill the non-common categories with 0

In [33]:
X_train['Carrier Name'] = X_train['Carrier Name Enc'].map(common_category_map).fillna(0).astype(int)
X_val['Carrier Name'] = X_val['Carrier Name Enc'].map(common_category_map).fillna(0).astype(int)
test['Carrier Name'] = test['Carrier Name Enc'].map(common_category_map).fillna(0).astype(int)

Encode de common categores using *Count Encoding*

In [ ]:
X_train, X_val, test = p.encode(X_train, X_val, test, 'Carrier Name', 'count')

**Carrier Type**

After grouping the *5.* categories we decided to encode them in 2 distinct ways, and choose the best option in feature selection.

Starting with *Count Encoding*

In [ ]:
X_train, X_val, test = p.encode(X_train, X_val, test, 'Carrier Type', 'count')

And *One-Hot-Encoding*

In [ ]:
X_train, X_val, test = p.encode(X_train, X_val, test, 'Carrier Type', 'OHE')

**County of Injury**

As County of Injury has a considerable amount of unique values, it will be encoded using Count Encoder.

In [ ]:
X_train, X_val, test = p.encode(X_train, X_val, test, 'County of Injury', 'count')

**COVID-19 Indicator**

As this variable only has 2 categories, they will be encoded as follows:
- N - 0
- Y - 1

In [ ]:
print(X_train['COVID-19 Indicator Enc'].value_counts())
print(' ')
print(test['COVID-19 Indicator Enc'].value_counts())

In [48]:
X_train['COVID-19 Indicator'] = X_train['COVID-19 Indicator Enc'].replace({'N': 0, 'Y': 1})
X_val['COVID-19 Indicator'] = X_val['COVID-19 Indicator Enc'].replace({'N': 0, 'Y': 1})
test['COVID-19 Indicator'] = test['COVID-19 Indicator Enc'].replace({'N': 0, 'Y': 1})

**District Name**

As this variable has 8 unique values, Count Encoder will be used.

In [ ]:
print(X_train['District Name'].value_counts())
print(' ')
print(test['District Name'].value_counts())

In [ ]:
X_train, X_val, test = p.encode(X_train, X_val, test, 'District Name', 'count')

**Gender**

Since after grouping we only have 3 categories, we will also use *One-Hot-Encoding* and decide which is better in feature selection.

In [ ]:
X_train, X_val, test = p.encode(X_train, X_val, test, 'Gender', 'OHE')

**Medical Fee Region**

Even though this variable only contains 5 unique values, it is not clear whether there is an order between them or not. Therefore we will use *Count Encoding*

In [ ]:
print(df['Medical Fee Region'].value_counts())
print(' ')
print(test['Medical Fee Region'].value_counts())

In [ ]:
X_train, X_val, test = p.encode(X_train, X_val, test, 'Medical Fee Region', 'count')

**Industry Sector**

Since it contains too many categories to use OHE, we will use *Count Encoding*

In [ ]:
X_train, X_val, test = p.encode(X_train, X_val, test, 'Industry Sector', 'count')

Remove Encoded Variables

In [55]:
drop = ['Alternative Dispute Resolution', 'Attorney/Representative', 'Carrier Type', 'County of Injury',
        'COVID-19 Indicator', 'District Name', 'Gender', 'Carrier Name',
        'Medical Fee Region', 'Industry Sector']

In [ ]:
X_train.drop(columns = drop, axis = 1, inplace = True)
X_val.drop(columns = drop, axis = 1, inplace = True)
test.drop(columns = drop, axis = 1, inplace = True)

# 2.2 Missing Values

<a href="#top">Top &#129033;</a>

In [ ]:
X_train.isna().sum()

### 2.2.1 Dealing with Missing Values

In this subsection we will use the existence of missing values to create new features

**C-3 Date**

Create a Binary variable:
- 0 if C-3 date is missing
- 1 if C-3 date exists

In [ ]:
X_train['C-3 Date Binary'] = X_train['C-3 Date'].notna().astype(int)
X_val['C-3 Date Binary'] = X_val['C-3 Date'].notna().astype(int)
test['C-3 Date Binary'] = test['C-3 Date'].notna().astype(int)

**First Hearing Date**

Create a Binary variable:
- 0 if First Hearing Date is missing
- 1 if First Hearing Date exists

In [ ]:
X_train['First Hearing Date Binary'] = X_train['First Hearing Date'].notna().astype(int)
X_val['First Hearing Date Binary'] = X_val['First Hearing Date'].notna().astype(int)
test['First Hearing Date Binary'] = test['First Hearing Date'].notna().astype(int)

Remove transformed features.

In [60]:
drop = ['C-3 Date', 'First Hearing Date']

In [ ]:
X_train.drop(columns = drop, axis = 1, inplace = True)
X_val.drop(columns = drop, axis = 1, inplace = True)
test.drop(columns = drop, axis = 1, inplace = True)

### 2.2.2 Filling Missing Values

In this subsection we will deal with missing values by filling them with constants, statistical methods and using predictive models.

**IME-4 Count**

Since IME-4 Count represents the number of IME-4 forms received per claim, we considered that a missing value represented 0 received forms, hence we will fill them with 0.

In [62]:
X_train['IME-4 Count'] = X_train['IME-4 Count'].fillna(0)
X_val['IME-4 Count'] = X_val['IME-4 Count'].fillna(0)
test['IME-4 Count'] = test['IME-4 Count'].fillna(0)

**Industry Code**

Assuming that a missing value in Industry Code represents an unknown code, it will be filled with 0.

In [63]:
X_train['Industry Code'] = X_train['Industry Code'].fillna(0)
X_val['Industry Code'] = X_val['Industry Code'].fillna(0)
test['Industry Code'] = test['Industry Code'].fillna(0)

**Accident Date** & **C-2 Date**

Fill Year, Month and Day with median. Then recompute full date and from there fill missing values in Day of Week

In [64]:
p.fill_dates(X_train, [X_val, test], 'Accident Date')
p.fill_dates(X_train, [X_val, test], 'C-2 Date')

Identify missing values and recompute Accident Date and C-2 Date to fill Day of the week

In [65]:
p.fill_dow([X_train, X_val, test], 'Accident Date')
p.fill_dow([X_train, X_val, test], 'C-2 Date')

**Time Between**

Fill missing values with the recomputed times using the medians computed above.

In [ ]:
X_train = p.fill_missing_times(X_train, ['Accident to Assembly Time', 
                             'Assembly to C-2 Time',
                             'Accident to C-2 Time'])

X_val = p.fill_missing_times(X_val, ['Accident to Assembly Time', 
                             'Assembly to C-2 Time',
                             'Accident to C-2 Time'])

test = p.fill_missing_times(test, ['Accident to Assembly Time', 
                             'Assembly to C-2 Time',
                             'Accident to C-2 Time'])

**Birth Year**

To fill the missing values, we will start by creating a mask, which filters for observations where `Age at Injury` and `Accident Date Year` are not missing, and when `Birth Year` is either missing or zero. Since we are going to use `Age at Injury` and `Accident Date Year` to compute `Birth Year`, ensuring those two variables are no missing is crucial. Then, we also decided to recompute the `Birth Year` when it is 0, since it is impossible to have 0 as a `Birth Year`.

In [36]:
p.fill_birth_year([X_train, X_val, test])

**Average Weekly Wage**

In [37]:
p.ball_tree_impute([X_train, X_val, test], 'Average Weekly Wage')

Having treated all missing values, we will create one last feature

**Wage to Age Ratio**

In [38]:
X_train['Wage to Age Ratio'] = np.where(X_train['Age at Injury'] != 0,
                                         X_train['Average Weekly Wage'] / X_train['Age at Injury'],
                                         0)

X_val['Wage to Age Ratio'] = np.where(X_val['Age at Injury'] != 0,
                                         X_val['Average Weekly Wage'] / X_val['Age at Injury'],
                                         0)

test['Wage to Age Ratio'] = np.where(test['Age at Injury'] != 0,
                                         test['Average Weekly Wage'] / test['Age at Injury'],
                                         0)

# 2.3 Outliers

<a href="#top">Top &#129033;</a>

### Outlier Detection

To detect outliers we will use a function that plots boxplots and identifies outliers based on the Interquartile Range method. This function will also add to a list all columns with a higher percentage of outliers than a previously set threshold.

In [ ]:
p.detect_outliers_iqr(X_train, 0.0001)

### Dealing With Outliers


**Average Weekly Wage**

Apply a square root transformation

In [41]:
X_train['Average Weekly Wage Sqrt'] = np.sqrt(X_train['Average Weekly Wage'])

X_val['Average Weekly Wage Sqrt'] = np.sqrt(X_val['Average Weekly Wage'])

test['Average Weekly Wage Sqrt'] = np.sqrt(test['Average Weekly Wage'])

Winsorization for `Average Weekly Wage`

In [40]:
upper_limit = X_train['Average Weekly Wage'].quantile(0.99)
lower_limit = X_train['Average Weekly Wage'].quantile(0.01)


X_train['Average Weekly Wage'] = X_train['Average Weekly Wage'].clip(lower = lower_limit
                                                                  , upper=upper_limit)

**Birth Year**

In [42]:
lower_limit = X_train['Birth Year'].quantile(0.01)

X_train['Birth Year'] = X_train['Birth Year'].clip(lower = lower_limit)

**IME-4 Count**

In [43]:
X_train['IME-4 Count Log'] = np.log1p(X_train['IME-4 Count'])
X_train['IME-4 Count Double Log'] = np.log1p(X_train['IME-4 Count Log'])

X_val['IME-4 Count Log'] = np.log1p(X_val['IME-4 Count'])
X_val['IME-4 Count Double Log'] = np.log1p(X_val['IME-4 Count Log'])

test['IME-4 Count Log'] = np.log1p(test['IME-4 Count'])
test['IME-4 Count Double Log'] = np.log1p(test['IME-4 Count Log'])

**Accident Date Year**

In [44]:
lower_limit = X_train['Accident Date Year'].quantile(0.01)

X_train['Accident Date Year'] = X_train['Accident Date Year'].clip(lower = lower_limit)

**C-2 Date Year**

In [45]:
lower_limit = X_train['C-2 Date Year'].quantile(0.01)

X_train['C-2 Date Year'] = X_train['C-2 Date Year'].clip(lower = lower_limit)

## 2.4 Scaling

Ensure y_train has the same indices as X_train

In [46]:
y_train = y_train[X_train.index]

**Variable type split**

In [31]:
num = ['Age at Injury', 'Average Weekly Wage', 'Birth Year',
       'IME-4 Count', 'Number of Dependents', 'Accident Date Year',
       'Accident Date Month', 'Accident Date Day', 
       'Assembly Date Year', 'Assembly Date Month', 
       'Assembly Date Day', 'C-2 Date Year', 'C-2 Date Month',
       'C-2 Date Day', 'Accident to Assembly Time',
       'Assembly to C-2 Time', 'Accident to C-2 Time', 
       'Wage to Age Ratio', 'Average Weekly Wage Sqrt',
       'IME-4 Count Log', 'IME-4 Count Double Log']

categ = [var for var in X_train.columns if var not in num]

categ_count_encoding = ['Carrier Name Enc', 'Carrier Type Enc',
                        'County of Injury Enc', 'District Name Enc',
                        'Medical Fee Region Enc', 'Industry Sector Enc']


categ_label_bin = [var for var in X_train.columns if var
                   in categ and var not in categ_count_encoding]


**Scaling**

In [ ]:
robust = RobustScaler()

In [ ]:
# Scaling the numerical features in the training set using RobustScaler
X_train_num_RS = robust.fit_transform(X_train[num])
X_train_num_RS = pd.DataFrame(X_train_num_RS, columns=num, index=X_train.index)

In [ ]:
# Scaling the numerical features in the validation set using the fitted RobustScaler
X_val_num_RS = robust.transform(X_val[num])
X_val_num_RS = pd.DataFrame(X_val_num_RS, columns=num, index=X_val.index)

# Scaling the numerical features in the test set using the same fitted RobustScaler
test_num_RS = robust.transform(test[num])
test_num_RS = pd.DataFrame(test_num_RS, columns=num, index=test.index)

** ** 

Having performed all data transformations, we will export the data to later use it in the Modelling section of this notebook. This will allow to save time by not always having to run computationally expensive methods such as RFE.

To go to the Modelling section click in the button.

[Go to Modeling &#129034;](#modeling)

<a class="anchor" id="feature-selection"></a>



In [46]:
# X_train.to_csv('./data/X_train_treated.csv')
# X_val.to_csv('./data/X_val_treated.csv')
# y_train.to_csv('./data/y_train_treated.csv')
# y_val.to_csv('./data/y_val_treated.csv')
# test.to_csv('./data/test_treated.csv')

** **

# 3. Feature Selection

<a href="#top">Top &#129033;</a>

In this section we will perform Feature Selection. Having already split our variables into *Numeric*, *Categorical*, *Categorical encoded using Count Encoding* and *Categorical (other)*, we will apply the following methods:

<br>

| Method                        | Feature Types |
| ----------------------------- | ------------- |
| Variance Threshold            | All           |
| Correlation                   | Numerical     |
| Chi-Square Test               | Categorical   |
| Mutual Information            | Categorical   |
| RFE                           | All           |
| LASSO                         | Numerical     |
| Tree-Based Feature Importance | All           |


## 3.1 Filter-Based Methods

<a href="#top">Top &#129033;</a>

Filter-based methods evaluate the relevance of features independently of the model using statistical measures like correlation, Chi-square tests, and mutual information. This section explores how these methods help reduce dimensionality, improve model performance, and prevent overfitting by selecting the most informative features.



**Variance Threshold**

In [ ]:
X_train[num].var() 

**Spearman Correlation Matrix**

In [ ]:
fs.correlation_matrix(X_train[num])

**Chi Squared Test**

In [ ]:
fs.chi_squared(X_train[categ_label_bin], y_train)

**Mutual Information Test**

In [ ]:
fs.mutual_info(X_train[categ], y_train, threshold = 0.05)

## 3.2 Wrapper Methods

<a href="#top">Top &#129033;</a>

Unlike filter methods, which assess features independently, wrapper methods evaluate the effectiveness of feature subsets by measuring the model’s performance, making them more computationally expensive but often more accurate in selecting relevant features.

**Recursive Feature Elimination (RFE) - Logistic Regression**

In [ ]:
n_features = np.arange(10, len(X_train.columns) + 1)
model = LogisticRegression()
rfe(X_train, y_train, X_val, y_val, 
    n_features = n_features, 
    model = model)

In [ ]:
import play_song as p
p.play_('audio.mp3')

**Recursive Feature Elimination (RFE) - Random Forest Classifier**

In [ ]:
# Perform Recursive Feature Elimination (RFE) to select the top features based on their importance for a logistic regression model
n_features = np.arange(2, len(X_train[num].columns) + 1)
model = RandomForestClassifier()
fs.rfe(X_train[num], y_train, n_features = n_features, model = model)

# 

## 3.3 Embedded Methods

<a href="#top">Top &#129033;</a>

These methods use algorithms that inherently select features as part of the model’s learning process. Embedded methods are computationally efficient and tend to be more accurate than filter methods, as they consider feature interactions and model performance simultaneously.


**Least Absolute Shrinkage and Selection Operator (LASSO)**

In [ ]:
fs.lasso(X_train[num], y_train, alpha = 0.01)

In [ ]:
fs.lasso(X_train[categ], y_train, alpha = 0.01)

**Extra Trees Classifier**

In [ ]:
fs.plot_feature_importance(X_train[num], X_train[categ], y_train, 
                        n_estimators = 250)

[Go to Modeling &#129034;](#modeling)

## 3.4 Final Features

<a href="#top">Top &#129033;</a>

**Final Decision**

<br>

`Numeric Variables`
<br><br>


| Variable                   | Variance | Correlation | RFE LR | RFE RF | Lasso | ExtraTrees | `Decision` |
| -------------------------- | -------- | ----------- | ------ | ------ | ----- | ---------- | ---------- |
| Accident Date Day          |    K     |             |        |        |   D   |            |            |
| Accident Date Month        |    K     |             |        |        |   D   |            |            |
| Accident Date Year         |    K     |    HC_4     |        |        |   D   |            |            |
| Accident to Assembly Time  |    K     |             |        |        |   K   |            |            |
| Accident to C-2 Time       |    K     |             |        |        |   K   |            |            |
| Age at Injury              |    K     |    HC_3     |        |        |   K   |            |            |
| Assembly Date Day          |    K     |             |        |        |   D   |            |            |
| Assembly Date Month        |    K     |             |        |        |   D   |            |            |
| Assembly Date Year         |    K     |    HC_4     |        |        |   K   |            |            |
| Assembly to C-2 Time       |    K     |             |        |        |   K   |            |            |
| Average Weekly Wage        |    K     |    HC_2     |        |        |   K   |            |            |
| Average Weekly Wage Sqrt   |    K     |    HC_2     |        |        |   K   |            |            |
| Birth Year                 |    K     |    HC_3     |        |        |   K   |            |            |
| C-2 Date Day               |    K     |             |        |        |   K   |            |            |
| C-2 Date Month             |    K     |             |        |        |   K   |            |            |
| C-2 Date Year              |    K     |    HC_4     |        |        |   D   |            |            |
| IME-4 Count                |    K     |    HC_1     |        |        |   K   |            |            |
| IME-4 Count Double Log     |    K     |    HC_1     |        |        |   D   |            |            |
| IME-4 Count Log            |    K     |    HC_1     |        |        |   K   |            |            |
| Number of Dependents       |    K     |             |        |        |   D   |            |            |
| Wage to Age Ratio          |    K     |             |        |        |   K   |            |            |

<br>

`Categorical Variables`
<br><br>


| Variable                                | Chi-Squared | MI | RFE LR | RFE RF | Lasso | Extra trees | `Decision` |
| --------------------------------------- | ----------- | -- | ------ | ------ | ----- | ----------- | ---------- |
| Accident Date Day of Week               |      D      | D  |        |        |   K   |             |            |
| Assembly Date Day of Week               |      D      | D  |        |        |   K   |             |            |
| Age Group                               |      K      | K  |        |        |   K   |             |            |
| Alternative Dispute Resolution Bin      |      K      | D  |        |        |   K   |             |            |
| Attorney/Representative Bin             |      K      | K  |        |        |   K   |             |            |
| Carrier Name Enc                        |      K      | K  |        |        |   K   |             |            |
| Carrier Type Enc                        |      NA     | D  |        |        |   K   |             |            |
| Carrier Type_2A. SIF                    |      K      | D  |        |        |   D   |             |            |
| Carrier Type_3A. SELF PUBLIC            |      K      | D  |        |        |   D   |             |            |
| Carrier Type_4A. SELF PRIVATE           |      K      | D  |        |        |   D   |             |            |
| Carrier Type_5. SPECIAL FUND OR UNKNOWN |      K      | D  |        |        |   D   |             |            |
| C-2 Date Day of Week                    |      K      | D  |        |        |   K   |             |            |
| C-3 Date Binary                         |      K      | K  |        |        |   K   |             |            |
| COVID-19 Indicator Enc                  |      K      | D  |        |        |   K   |             |            |
| County of Injury Enc                    |      NA     | D  |        |        |   K   |             |            |
| District Name Enc                       |      NA     | D  |        |        |   K   |             |            |
| First Hearing Date Binary               |      K      | K  |        |        |   K   |             |            |
| Gender Enc                              |      K      | D  |        |        |   K   |             |            |
| Gender_M                                |      K      | D  |        |        |   D   |             |            |
| Gender_U/X                              |      K      | D  |        |        |   D   |             |            |
| Industry Code                           |      K      | K  |        |        |   K   |             |            |
| Industry Sector Enc                     |      NA     | D  |        |        |   K   |             |            |
| Insurance                               |      K      | D  |        |        |   D   |             |            |
| Medical Fee Region Enc                  |      NA     | D  |        |        |   K   |             |            |
| WCIO Cause of Injury Code               |      K      | K  |        |        |   K   |             |            |
| WCIO Codes                              |      K      | K  |        |        |   K   |             |            |
| WCIO Nature of Injury Code              |      K      | K  |        |        |   K   |             |            |
| WCIO Part Of Body Code                  |      K      | K  |        |        |   K   |             |            |
| Zip Code Valid                          |      D      | D  |        |        |   D   |             |            |



| Symbol | Meaning |
| ------ | ------- |
| K      | Keep    |
| D      | Discard |
| HC_*N*   | High Correlation Identifier |
| NA     | Not Applicable |


# 4. Modeling

<a class="anchor" id="modeling"></a>

[Go to Feature Selection &#129034;](#feature-selection)

<a href="#top">Top &#129033;</a>

Start by importing the correct datasets.


In [3]:
X_train = pd.read_csv('./data/X_train_treated.csv', index_col = 'Claim Identifier')
X_val = pd.read_csv('./data/X_val_treated.csv', index_col = 'Claim Identifier')
y_train = pd.read_csv('./data/y_train_treated.csv', index_col = 'Claim Identifier')
y_val = pd.read_csv('./data/y_val_treated.csv', index_col = 'Claim Identifier')
test = pd.read_csv('./data/test_treated.csv', index_col = 'Claim Identifier')

And select the columns to use for predictions purposes

In [ ]:
columns = []

X_train_filtered = X_train[columns]
X_val_filtered = X_val[columns]
test_filtered = test[columns]

In [27]:
from sklearn.metrics import f1_score, precision_score, recall_score


def run_model(model_name, X, y):
    
    if model_name == 'LR':
        model = LogisticRegression().fit(X, y)
    elif model_name == 'SGD':
        model = SGDClassifier().fit(X, y)
    elif model_name == 'DT':
        model = DecisionTreeClassifier().fit(X, y)
    elif model_name == 'RF':
        model = RandomForestClassifier().fit(X, y)
    elif model_name == 'AdaBoost':
        model = AdaBoostClassifier().fit(X, y)
    elif model_name == 'GBoost':
        model = GradientBoostingClassifier().fit(X, y)
    elif model_name == 'XGB':
        model = XGBClassifier().fit(X, y)
    elif model_name == 'MLP':
        model = MLPClassifier().fit(X, y)
        
    return model

def modeling(model_names, X_train, y_train, X_val, y_val):
    
    results = {}
    
    for model_name in model_names:
        print(f"Training model: {model_name}")
        
        # Train model
        model = run_model(model_name, X_train, y_train)
        
        # Predict on train and validation data
        y_train_pred = model.predict(X_train)
        y_val_pred = model.predict(X_val)
        
        # Calculate metrics for the training set
        train_precision = precision_score(y_train, y_train_pred, average='macro')
        train_recall = recall_score(y_train, y_train_pred, average='macro')
        train_f1 = f1_score(y_train, y_train_pred, average='macro')
        
        # Calculate metrics for the validation set
        val_precision = precision_score(y_val, y_val_pred, average='macro')
        val_recall = recall_score(y_val, y_val_pred, average='macro')
        val_f1 = f1_score(y_val, y_val_pred, average='macro')
        
        # Save results
        results[model_name] = {
            'train_precision': train_precision,
            'val_precision': val_precision,
            'train_recall': train_recall,
            'val_recall': val_recall,
            'train_macro_f1': train_f1,
            'val_macro_f1': val_f1
        }
        
        print(f"{model_name} - Train: Precision: {train_precision:.4f}, Recall: {train_recall:.4f}, Macro F1: {train_f1:.4f}")
        print(f"{model_name} - Validation: Precision: {val_precision:.4f}, Recall: {val_recall:.4f}, Macro F1: {val_f1:.4f}\n")
    
    return results


In [ ]:
# Define the models to run
model_names = ['LR', 'DT', 'RF', 'AdaBoost','XGB', 'MLP']

results = modeling(model_names, X_train, y_train, X_val, y_val)

In [ ]:
import play_song as s
s.play_('audio.mp3')

In [ ]:
results_df = pd.DataFrame(results).T
results_df